# Generating Randomized Data

In [1]:
import pandas as pd
import numpy as np

rows = 100

# Training Set

# 1 is true positive, 0 is false postive
labels = np.random.randint(2, size=rows)

# feature values drawn from a normal distribution
# with mean=0 and var=1 
feature1 = np.random.randn(rows)
feature2 = np.random.randn(rows)
feature3 = np.random.randn(rows)
feature4 = np.random.randn(rows)
feature5 = np.random.randn(rows)

feature_dict = {'feature1': feature1,'feature2': feature2, 
                'feature3': feature3, 'feature4': feature4, 
                'feature5': feature5}

mutect_output = pd.DataFrame(feature_dict)

# Test Set

# 1 is true positive, 0 is false postive
test_labels = np.random.randint(2, size=rows)

# feature values drawn from a normal distribution
# with mean=0 and var=1 
test_feature1 = np.random.randn(rows)
test_feature2 = np.random.randn(rows)
test_feature3 = np.random.randn(rows)
test_feature4 = np.random.randn(rows)
test_feature5 = np.random.randn(rows)

test_feature_dict = {'feature1': test_feature1,'feature2': test_feature2, 
                'feature3': test_feature3, 'feature4': test_feature4, 
                'feature5': test_feature5}

test_mutect_output = pd.DataFrame(test_feature_dict)



# Gameplan

We are going to build a classifier to separate mutect outputs between true positive and false positive. It is unclear which approach will yield the best results, so we will try a few. 
1. Logisitic Regression
2. K-Nearest Neighbors
3. Neural Network

more ideas to come...



In [2]:
import sklearn
from sklearn import linear_model, datasets

X = mutect_output.as_matrix(columns=None)
Y = labels

logreg = linear_model.LogisticRegression(C=1e5)
logreg.fit(X, Y)

X_test = test_mutect_output.as_matrix(columns=None)
Y_test = test_labels

print logreg.score(X_test, Y_test)


0.49


## Code for Cross Validation

In [3]:
##### TO BE COMPLETED LATER
a = pd.Series([1,2,3,4,5])
print(a.sum())

15


## Read in the Dream3 Files

In [4]:
data_truth = pd.read_csv('DREAM3_truth.txt', delimiter='\t', low_memory=False)
data =  pd.read_csv('DREAM3_unfiltered.txt', delimiter='\t', low_memory=False)
data['Label'] = 0
print list(data)

# create labels for the data
"""

for row in range(200): #len(data)):
    chrom = data.loc[:,('CHROM', row)]
    pos = data.loc[:,('POS', row)] 
    ref = data.loc[:,('REF', row)]
    alt = data.loc[:,('ALT', row)]
    ans = ((data_truth['CHROM'] == chrom) & (data_truth['POS'] == pos) & 
                (data_truth['REF'] == ref) & (data_truth['ALT'] == alt)).any()
    if ans:
        data['Label'][row] = 1


a = data['Label']
print(a.sum())
"""

['CHROM', 'POS', 'REF', 'ALT', 'AC', 'AF', 'AN', 'BaseQRankSum', 'ClippingRankSum', 'DP', 'FS', 'MQRankSum', 'NLOD', 'N_ART_LOD', 'RAW_MQ', 'ReadPosRankSum', 'SOR', 'TLOD', 'TLOD_FWD', 'TLOD_REV', 'TUMOR_SB_POWER_FWD', 'TUMOR_SB_POWER_REV', 'G15512.prenormal.sorted.ALT_F1R2', 'G15512.prenormal.sorted.ALT_F2R1-GF', 'G15512.prenormal.sorted.FOXOG', 'G15512.prenormal.sorted.QSS', 'G15512.prenormal.sorted.REF_F1R2', 'G15512.prenormal.sorted.REF_F2R1', 'IS3.snv.indel.sv.ALT_F1R2', 'IS3.snv.indel.sv.ALT_F2R1-GF', 'IS3.snv.indel.sv.FOXOG', 'IS3.snv.indel.sv.QSS', 'IS3.snv.indel.sv.REF_F1R2', 'IS3.snv.indel.sv.REF_F2R1', 'Label']


"\n\nfor row in range(200): #len(data)):\n    chrom = data.loc[:,('CHROM', row)]\n    pos = data.loc[:,('POS', row)] \n    ref = data.loc[:,('REF', row)]\n    alt = data.loc[:,('ALT', row)]\n    ans = ((data_truth['CHROM'] == chrom) & (data_truth['POS'] == pos) & \n                (data_truth['REF'] == ref) & (data_truth['ALT'] == alt)).any()\n    if ans:\n        data['Label'][row] = 1\n\n\na = data['Label']\nprint(a.sum())\n"

## Dream4 Data 
Labeled through excel

In [5]:
import pandas as pd
import numpy as np
data = pd.read_excel('dream4_data.xlsx')

In [6]:
#data['synthetic.challenge.set4.normal.FOXOG']
#print data
#print data.dropna()
#print data['synthetic.challenge.set4.normal.FOXOG'][0:100]

In [7]:
print len(data)
data.dropna(inplace=True)
print len(data)

def normalize_value(df, column, mean, std):
    name = column + '_norm'
    df[name] = (df[column] - mean) / std
    return df
    
def build_summary(df, columns):
    """
    df is a dataframe
    columns is a list of columns get summary data
    returns a dict with column as key - value is (median, std)
    """
    summary_dict = dict()
    for name in columns:
        mean = np.mean(data[name])
        std = np.std(data[name])
        summary_dict[name] = (mean, std)
    return summary_dict

def normalize_cols(df, columns):
    col_dict = build_summary(df, columns)
    for col in columns:
        stats = col_dict[col]
        normalize_value(df, col, stats[0], stats[1])
    return df
cols = ['DP', 'FS', 'RAW_MQ', 'TLOD_FWD', 'TLOD_REV', 'BaseQRankSum', 'ClippingRankSum', 'MQRankSum', 'ReadPosRankSum']
normalize_cols(data, cols)


print data['DP_norm'][0:10]


43923
43688
0   -0.262523
1   -0.186452
2   -0.167857
3   -0.164476
4   -0.167857
5   -0.159405
6   -0.159405
7   -0.156024
8   -0.198285
9   -0.156024
Name: DP_norm, dtype: float64


In [8]:
Y = data['Label']
del data['Label']

barrier = int(len(data) * (3.0/10.0))

data_test = data[:barrier]
Y_test = Y[:barrier]
data = data[barrier:]
Y = Y[barrier:]

X_base = np.random.randint(2, size=len(data)).reshape(-1, 1)
X_base_test = np.random.randint(2, size=len(data_test)).reshape(-1, 1)

X_1 = data.as_matrix(columns=['DP', 'FS'])
X_1_test = data_test.as_matrix(columns=['DP', 'FS'])

X_2 = data.as_matrix(columns=['DP', 'FS', 'RAW_MQ'])
X_2_test = data_test.as_matrix(columns=['DP', 'FS', 'RAW_MQ'])

X_3 = data.as_matrix(columns=['DP', 'FS', 'RAW_MQ','TLOD_FWD', 'TLOD_REV'])
X_3_test = data_test.as_matrix(columns=['DP', 'FS', 'RAW_MQ', 'TLOD_FWD', 'TLOD_REV'])

X_4 = data.as_matrix(columns=['DP', 'FS','TLOD_FWD', 'TLOD_REV'])
X_4_test = data_test.as_matrix(columns=['DP', 'FS', 'TLOD_FWD', 'TLOD_REV'])

X_5 = data.as_matrix(columns=['TLOD_FWD', 'TLOD_REV'])
X_5_test = data_test.as_matrix(columns=['TLOD_FWD', 'TLOD_REV'])

X_6 = data.as_matrix(columns=['DP_norm', 'FS_norm', 'RAW_MQ_norm','TLOD_FWD_norm', 'TLOD_REV_norm'])
X_6_test = data_test.as_matrix(columns=['DP_norm', 'FS_norm', 'RAW_MQ_norm', 'TLOD_FWD_norm', 'TLOD_REV_norm'])

X_7 = data.as_matrix(columns=['DP_norm', 'FS_norm'])
X_7_test = data_test.as_matrix(columns=['DP_norm', 'FS_norm'])

X_8 = data.as_matrix(columns=['DP_norm', 'FS_norm', 'RAW_MQ_norm','TLOD_FWD_norm', 'TLOD_REV_norm', 'BaseQRankSum_norm', 'ClippingRankSum_norm', 'MQRankSum_norm', 'ReadPosRankSum_norm'])
X_8_test = data_test.as_matrix(columns=['DP_norm', 'FS_norm', 'RAW_MQ_norm', 'TLOD_FWD_norm', 'TLOD_REV_norm', 'BaseQRankSum_norm', 'ClippingRankSum_norm', 'MQRankSum_norm', 'ReadPosRankSum_norm'])


#X_1 = data.as_matrix(columns=None)
#X_3 = data.as_matrix(columns=['BaseQRankSum', 'ClippingRankSum', 'DP', 'FS', 'MQRankSum', 'N_ART_LOD', 'RAW_MQ', 
#                              'ReadPosRankSum', 'SOR','TLOD', 'TLOD_FWD', 'TLOD_REV'])


In [9]:

logreg = linear_model.LogisticRegression(C=1)
logreg.fit(X_base, Y)
print logreg.score(X_base_test, Y_test)

logreg.fit(X_1, Y)
print logreg.score(X_1_test, Y_test)

logreg.fit(X_2, Y)
print logreg.score(X_2_test, Y_test)

logreg.fit(X_3, Y)
print logreg.score(X_3_test, Y_test)

logreg.fit(X_4, Y)
print logreg.score(X_4_test, Y_test)

logreg.fit(X_5, Y)
print logreg.score(X_5_test, Y_test)

logreg.fit(X_6, Y)
print logreg.score(X_6_test, Y_test)

logreg.fit(X_7, Y)
print logreg.score(X_7_test, Y_test)

logreg.fit(X_8, Y)
print logreg.score(X_8_test, Y_test)


0.491149092019
0.672974210285
0.508850907981
0.508850907981
0.726079658172
0.583091713719
0.751564169083
0.673203113078
0.831069739051


In [10]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

scores = cross_val_score(logreg, X_8, Y, cv=7)
print scores
print scores.mean(), scores.std()

[ 0.814374    0.79262989  0.80178531  0.76974136  0.83245594  0.85122454
  0.80082418]
0.809005029728 0.0247562097701


In [11]:
##### SVM Classifier
from sklearn.svm import SVC

### POLY KERNEL (DEGREE 3)
svc = SVC(kernel='poly')
svc.fit(X_1, Y)
print svc.score(X_1_test, Y_test)

svc.fit(X_2, Y)
print svc.score(X_2_test, Y_test)

svc.fit(X_3, Y)
print svc.score(X_3_test, Y_test)

svc.fit(X_4, Y)
print svc.score(X_4_test, Y_test)

svc.fit(X_5, Y)
print svc.score(X_5_test, Y_test)

svc.fit(X_6, Y)
print svc.score(X_6_test, Y_test)

svc.fit(X_7, Y)
print svc.score(X_7_test, Y_test)

svc.fit(X_8, Y)
print svc.score(X_8_test, Y_test)


"\n### POLY KERNEL (DEGREE 3)\nsvc = SVC(kernel='poly')\nsvc.fit(X_1, Y)\nprint svc.score(X_1_test, Y_test)\n\nsvc.fit(X_2, Y)\nprint svc.score(X_2_test, Y_test)\n\nsvc.fit(X_3, Y)\nprint svc.score(X_3_test, Y_test)\n\n#svc.fit(X_4, Y)\n#print svc.score(X_4_test, Y_test)\n\n#svc.fit(X_5, Y)\n#print svc.score(X_5_test, Y_test)\n\nsvc.fit(X_6, Y)\nprint svc.score(X_6_test, Y_test)\n\n#svc.fit(X_7, Y)\n#print svc.score(X_7_test, Y_test)\n\nsvc.fit(X_8, Y)\nprint svc.score(X_8_test, Y_test)\n"

In [12]:
### RBF KERNEL
svc = SVC(kernel='rbf')
svc.fit(X_1, Y)
print svc.score(X_1_test, Y_test)

svc.fit(X_2, Y)
print svc.score(X_2_test, Y_test)

svc.fit(X_3, Y)
print svc.score(X_3_test, Y_test)

svc.fit(X_4, Y)
print svc.score(X_4_test, Y_test)

svc.fit(X_5, Y)
print svc.score(X_5_test, Y_test)

svc.fit(X_6, Y)
print svc.score(X_6_test, Y_test)

svc.fit(X_7, Y)
print svc.score(X_7_test, Y_test)

svc.fit(X_8, Y)
print svc.score(X_8_test, Y_test)


0.717686555776
0.825576072028
0.801312376011
0.791088051274
0.731878528918
0.78284755074
0.657332519457
0.868457195178


In [13]:
scores = cross_val_score(svc, X_8, Y, cv=7)
print scores
print scores.mean(), scores.std()


[ 0.85214008  0.83428702  0.83680476  0.81689174  0.86472877  0.88326848
  0.82944139]
0.845366034771 0.021104564681


In [10]:
### Linear KERNEL 
svc = SVC(kernel='linear')
svc.fit(X_1, Y)
print svc.score(X_1_test, Y_test)

svc.fit(X_2, Y)
print svc.score(X_2_test, Y_test)

svc.fit(X_3, Y)
print svc.score(X_3_test, Y_test)

svc.fit(X_4, Y)
print svc.score(X_4_test, Y_test)

svc.fit(X_5, Y)
print svc.score(X_5_test, Y_test)

svc.fit(X_6, Y)
print svc.score(X_6_test, Y_test)

svc.fit(X_7, Y)
print svc.score(X_7_test, Y_test)

svc.fit(X_8, Y)
print svc.score(X_8_test, Y_test)

NameError: name 'SVC' is not defined